# Task 1

In [5]:
!pip install arxiv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import logging
logging.getLogger("arxiv").setLevel(logging.ERROR)   
import arxiv
import re
import os
import re
import requests
import json
import time
import threading
import queue



In [2]:
num = 0
student_ids = ['23127130', '23127051', '23127107']
for id in student_ids:
    for number in id:
        num += int(number)
        
print("Sum of digits in student IDs:", num)
k = num % 50
print("k value:", k)

Sum of digits in student IDs: 63
k value: 13


In [3]:
def is_valid(number:int, k:int) -> bool:
    sum = 0
    while number > 0:
        sum += number % 10
        number = int(number / 10)
    print(sum)
    if (sum % 50) == k:
        return True 
    return False

In [5]:
start_month = 3
start_year = 2023
start_ID = 7856
end_month = 4
end_year = 2023
end_ID = 4606


def is_valid(number:int, k:int) -> bool:
    sum = 0
    while number > 0:
        sum += number % 10
        number = int(number / 10)
    
    if (sum % 50) == k:
        return True 
    return False

def get_ID(month, year, number):
    """Return arXiv ID in YYMM.NNNNN format."""
    return f"{year % 100:02d}{month:02d}.{number:05d}"

def id_exists(paper_id):
    """Check if a specific arXiv ID exists."""
    search = arxiv.Search(id_list=[paper_id])
    client = arxiv.Client(page_size=1, delay_seconds=0.2)
    try:
        next(client.results(search))
        return True
    except StopIteration:
        return False
    except Exception:
        # Network or parsing error — assume not found for safety
        return False

def find_first_id(year, month):
    """Find the first valid arXiv ID of a given month using exponential + binary search."""
    low, high = 1, 1
    # Exponential search upward until we find a valid ID
    while not id_exists(get_ID(month, year, high)):
        high *= 2
        if high > 99999:
            return None  # No valid papers this month

    # Now binary search between low and high to find the *first* valid ID
    while low + 1 < high:
        mid = (low + high) // 2
        if id_exists(get_ID(month, year, mid)):
            high = mid
        else:
            low = mid
    return high

def find_last_id(year, month):
    """Find the last valid arXiv ID of a given month."""
    low, high = 1, 1
    # Exponential search upward until we find a missing ID
    while id_exists(get_ID(month, year, high)):
        high *= 2
        if high > 99999:
            return 99999
    low = high // 2
    # Binary search for the last existing ID
    while low + 1 < high:
        mid = (low + high) // 2
        if id_exists(get_ID(month, year, mid)):
            low = mid
        else:
            high = mid
    return low

def get_IDs_month(month, year, start_number, end_number):
    """Get all valid arXiv IDs in a given month."""
    id_month = []
    
    for i in range(start_number, end_number + 1):
        number = i
        if is_valid(number, k) == True:
            id_month.append(get_ID(month, year, i)) 
    return id_month

def get_IDs_All(start_month, start_year, start_ID, end_month, end_year, end_ID):
    """Get all valid arXiv IDs in the given range."""
    ids = []
    y, m = start_year, start_month
    n_start = start_ID
    n_end = None
    
    while True:
        if y == end_year and m == end_month:
            n_end = end_ID
        else:
            n_end = find_last_id(y, m)
            if n_end is None:
                n_end = 0  # No papers this month

        if n_start <= n_end:
            ids.extend(get_IDs_month(m, y, n_start, n_end))

        if y == end_year and m == end_month:
            break
        m += 1
        if m > 12:
            m, y = 1, y + 1
        n_start = find_first_id(y, m)  # reset numbering

    return ids


In [6]:
list_id = get_IDs_All(start_month, start_year, start_ID, end_month, end_year, end_ID)

In [7]:
def format_yymm_id(base_id: str) -> str:
    """'2303.07856' -> '2303-07856'"""
    return base_id.replace('.', '-')

def get_paper_references(arxiv_id, delay=2):
    """
    Fetch references for a paper from Semantic Scholar API.
    
    Args:
        arxiv_id: arXiv ID (format: YYMM.NNNNN or YYMM.NNNNNvN)
        retry: number of retry attempts
        delay: delay between retries in seconds
    
    Returns:
        list: List of references with detailed information
    """
    # Clean arxiv_id (remove version suffix if present)
    clean_id = re.sub(r'v\d+$', '', arxiv_id)
    url = f"https://api.semanticscholar.org/graph/v1/paper/arXiv:{clean_id}"
    params = {
        "fields": "references,references.title,references.authors,references.year,references.venue,references.externalIds,references.publicationDate, references.abstract, references.publicationVenue, references.referenceCount, references.influentialCitationCount, references.journal, references.embedding, references.fieldsOfStudy, references.publicationTypes, references.isOpenAccess, references.s2FieldsOfStudy, references.citationCount, references.corpusId"
    }

    while True:
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                return data.get("references", [])
            elif response.status_code == 429:
                print(f"  Rate limit hit. Waiting {delay}s before retry...")
                time.sleep(delay)
            elif response.status_code == 404:
                print(f"  Paper {arxiv_id} not found in Semantic Scholar")
                return []
            else:
                print(f"  API returned status {response.status_code}, retrying in {delay}s...")
                time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"  Request error: {e}, retrying in {delay}s...")
            time.sleep(delay)
 

def convert_to_references_dict(references):
    """
    Convert Semantic Scholar references to the required format:
    Dictionary with arXiv IDs as keys (in "yyyymm-id" format) for papers with arXiv IDs.
    For papers without arXiv IDs, use DOI or generate a unique key.
    
    Args:
        references: List of references from Semantic Scholar API
    
    Returns:
        dict: Dictionary with paper IDs as keys and metadata as values
    """
    result = {}
    non_arxiv_counter = 1
    
    for ref in references:
        # The reference data is directly in ref, not under "citedPaper"
        
        # Skip if reference is None or empty
        if not ref:
            continue
        
        # Extract external IDs (may be None)
        external_ids = ref.get("externalIds", {})
        if external_ids is None:
            external_ids = {}
        
        arxiv_id = external_ids.get("ArXiv", "")
        doi = external_ids.get("DOI", "")
        # Only keep references that have arXiv_id
        if not arxiv_id:
            continue
        
        # Determine the key for this reference
        if arxiv_id:
            # Use arXiv ID in yyyymm-id format
            key = format_yymm_id(arxiv_id)
        elif doi:
            # Use DOI as key (sanitize it)
            key = f"doi:{doi.replace('/', '_')}"
        else:
            # Generate a unique key for papers without arXiv ID or DOI
            title = ref.get("title", "")
            if title:
                # Use first word of title + counter
                first_word = re.sub(r'[^\w]', '', title.split()[0] if title.split() else "unknown")
                key = f"ref_{first_word[:20]}_{non_arxiv_counter}"
            else:
                key = f"ref_unknown_{non_arxiv_counter}"
            non_arxiv_counter += 1
        
        # Extract authors
        authors_list = ref.get("authors", [])
        authors = [author.get("name", "") for author in authors_list if author.get("name")]
        
        # Extract dates (use publicationDate if available)
        publication_date = ref.get("publicationDate", "")
        year = ref.get("year")
        
        # Extract corupt ids: corpusId
        corpusId = ref.get("corpusId", "")
        abstract = ref.get("abstract", "")
        referenceCount = ref.get("referenceCount", "")
        citationCount = ref.get("citationCount", "")
        influentialCitationCount = ref.get("influentialCitationCount", "")
        isOpenAccess = ref.get("isOpenAccess", "")
        fieldsOfStudy= ref.get("fieldsOfStudy", "")
        s2FieldsOfStudy = ref.get("s2FieldsOfStudy", [])
        publicationTypes = ref.get("publicationTypes", [])
        journal = ref.get("journal", "")
        embedding = ref.get("embedding", [])
        
        # If no publication date but have year, create an ISO-like format
        if not publication_date and year:
            publication_date = f"{year}-01-01"  # Use Jan 1st as placeholder
        
        # Build metadata dictionary with required fields
        metadata = {
            "title": ref.get("title", ""),
            "authors": authors,
            "submission_date": publication_date if publication_date else "",
            "revised_dates": [],  # Semantic Scholar doesn't provide revision history
            "corpusId" : corpusId,
            "abstract" : abstract,
            "referenceCount" : referenceCount,
            "citationCount" : citationCount,
            "influentialCitationCount" : influentialCitationCount,
            "isOpenAccess" : isOpenAccess,
            "fieldsOfStudy" : fieldsOfStudy,
            "s2FieldsOfStudy" : s2FieldsOfStudy,
            "publicationTypes" : publicationTypes,
            "journal" : journal,
            "embedding" : embedding,
        }
        
        # Add optional fields for reference
        if doi:
            metadata["doi"] = doi
        if arxiv_id:
            metadata["arxiv_id"] = arxiv_id
        if ref.get("venue"):
            metadata["venue"] = ref.get("venue")
        if year:
            metadata["year"] = year
        
        result[key] = metadata
    
    return result


def save_references(arxiv_id, paper_folder, verbose=True):
    """
    Fetch and save references for a paper version to both JSON and BibTeX formats.
    
    Args:
        arxiv_id: arXiv ID (e.g., "2304.07856v1")
        version_folder: Path to version folder (e.g., "data/2304.07856/v1/")
        verbose: Whether to print progress messages
    
    Returns:
        bool: True if successful, False otherwise
    """
    # Check if the folder exists, if not, create it
    if not os.path.exists(paper_folder):
        os.makedirs(paper_folder, exist_ok=True)

    if verbose:
        print(f"Fetching references for {arxiv_id}...")

    references = get_paper_references(arxiv_id)

    if not references:
        if verbose:
            print(f"  No references found for {arxiv_id}")
        json_path = os.path.join(paper_folder, "references.json")
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump({}, f, indent=2, ensure_ascii=False)
        return False

    json_path = os.path.join(paper_folder, "references.json")
    references_dict = convert_to_references_dict(references)
    try:
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(references_dict, f, indent=2, ensure_ascii=False)
        if verbose:
            print(f"  Saved {len(references_dict)} references to references.json")
    except Exception as e:
        print(f"  Error saving JSON: {e}")
        return False
    

def extract_references_for_paper(paper_id, base_data_dir="../data"):
    """
    Extract references for all versions of a paper.
    
    Args:
        paper_id: arXiv paper ID without version (e.g., "2304.07856")
        base_data_dir: Base directory containing data folders
    
    Returns:
        dict: Statistics about the extraction
    """
    paper_id_key = format_yymm_id(paper_id)
    paper_folder = os.path.join(base_data_dir, paper_id_key)
    if (os.path.exists(paper_folder) == False):
        os.makedirs(paper_folder)
    save_references(paper_id, os.path.join(paper_folder))


In [8]:
print(len(list_id))

764


In [ ]:
def reference_worker(download_queue, base_data_dir, delay=2):
    processed = 0
    while True:
        arxiv_id = download_queue.get()
        if arxiv_id is None:
            print(f"[Reference] Thread finished. Total papers processed: {processed}")
            break
        try:
            print(f"[Reference] Starting: {arxiv_id}")
            extract_references_for_paper(arxiv_id, base_data_dir)
            processed += 1
            print(f"[Reference] Finished: {arxiv_id} (Total: {processed})")
            time.sleep(delay)
        except Exception as e:
            print(f"[Reference] Error for {arxiv_id}: {e}")
            continue


base_data_dir = "data"
if os.path.exists(base_data_dir) == False:
    os.makedirs(base_data_dir, exist_ok=True)

REFERENCE_THREAD_COUNT = 2
start_time = time.time()
download_queue = queue.Queue()
for id in list_id:
    download_queue.put(id)


reference_threads = []
for _ in range(REFERENCE_THREAD_COUNT):
    t = threading.Thread(target=reference_worker, args=(download_queue, base_data_dir, 2))
    t.start()
    reference_threads.append(t)

    
for t in reference_threads:
    t.join()

end_time = time.time()
print(f"Pipeline complete. Total time: {end_time - start_time:.2f} seconds")


[Reference] Starting: 2303.08005
[Reference] Starting: 2303.08014
Fetching references for 2303.08005...
Fetching references for 2303.08014...


  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...  Rate limit hit. Waiting 2s before retry...

  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Saved 13 references to references.json
[Reference] Finished: 2303.08005 (Total: 1)
[Reference] Starting: 2303.08023
Fetching references for 2303.08023...
  Rate limit hit. Waiting 2s before retry...
  Saved 27 references to references.json
[Reference] Finished: 2303.08014 (Total: 1)
[Reference] Starting: 2303.08032
Fetching references for 2303.08032...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit hit. Waiting 2s before retry...
  Rate limit 